In [ ]:
# Check last executions
past_time = "3d"
client_name = "usdf_efd"

# M1M3 Bump Test Log Error Analysis and Measured Forces

## Overview

This notebook facilitates the analysis of M1M3 force actuator “Bump Test” logs and the visualization of actuator performance through a **Bokeh-based interactive app**.
It provides the following features:

- **Bump Test Log Analysis:**
   Queries and processes recent bump test logs from the EFD to identify failed actuators and extract relevant information, such as deviation type and measured force.

- **Execution Details:**
   Retrieves and displays script execution logs for the `check_actuators.py` script. Details include:
   - Execution start and end times.
   - Duration of execution.
   - Final process and script statuses.

- **Failure Summary Table:**
   Summarizes failed actuators with details on failure time, orientation, measured forces, and deviations.
   Includes clickable links for further diagnostics of force and following errors of individual actuators.

- **Visual Diagnostics:**
   Provides comprehensive visualizations, including:
   - Plots of positive and negative measured forces for failed actuators.
   - A spatial layout showing the distribution of failed actuators on the M1M3 system.

This integrated approach enables efficient troubleshooting and performance assessment of M1M3 actuators, supporting both real-time and historical data analysis workflows.


In [ ]:
## Setup and Imports

import asyncio
import base64
import io
import re
from collections import defaultdict
from datetime import datetime, timedelta

import ipywidgets as widgets
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.time import Time, TimeDelta
from IPython.display import HTML, display
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import Checkbox, ColumnDataSource, CustomJS, Div, Select
from lsst_efd_client import EfdClient
from lsst.ts.xml.tables.m1m3 import FATable
from matplotlib.lines import Line2D

import nest_asyncio

nest_asyncio.apply()

In [ ]:
## Utility Functions

### Converting Past Time Strings


def convert_to_hours(past_time):
    """
    Convert a string that can be either a number of hours (e.g. '6h')
    or a number of days (e.g. '3d') to total number of hours (int).
    """
    match = re.match(r"(\d+)([dh]?)", past_time)
    if match:
        value, unit = match.groups()
        value = int(value)
        if unit == "d":
            return value * 24
        else:
            return value
    else:
        raise ValueError(
            "Invalid time format. Please use a format "
            "like '6h' for hours or '3d' for days."
        )

In [ ]:
### Query and Filter Script Queue Logs


async def query_script_queue_logs(
    start_time_str: str, end_time_str: str, client_name="usdf_efd"
):
    """
    Queries the log messages related to the script queue from the EFD.

    Parameters
    ----------
    start_time_str : str
        Start time in ISO format, e.g. "2024-11-04T12:00:00".
    end_time_str : str
        End time in ISO format, e.g. "2024-11-04T13:00:00".
    client_name : str, optional
        Name of the EFD client. Defaults to "usdf_efd".

    Returns
    -------
    pd.DataFrame
        DataFrame with script queue logs in the given time window.
    """
    # Convert string times to astropy Time
    start = Time(start_time_str, format="isot", scale="utc")
    end = Time(end_time_str, format="isot", scale="utc")

    # Create the EFD client
    possible_clients = ["summit_efd", "usdf_efd"]
    if client_name not in possible_clients:
        print(f"Invalid client name. Possible clients: {possible_clients}")
        return None

    client = EfdClient(client_name)
    script_logs = await client.select_time_series(
        topic_name="lsst.sal.ScriptQueue.logevent_script",
        fields="*",
        start=start,
        end=end,
    )

    return script_logs


def filter_and_process_queue_logs(script_logs):
    """
    Filters and processes the script logs DataFrame for a specific script (maintel/m1m3/check_actuators.py).

    Parameters
    ----------
    script_logs : pd.DataFrame
        DataFrame containing the script logs.

    Returns
    -------
    pd.DataFrame
        Processed DataFrame containing only relevant logs for the check_actuators script.
    """
    processState_mapping = {
        0: "UNKNOWN",
        1: "LOADING",
        2: "CONFIGURED",
        3: "RUNNING",
        4: "DONE",
        5: "LOADFAILED",
        6: "CONFIGURE_FAILED",
        7: "TERMINATED",
    }

    scriptState_mapping = {
        0: "UNKNOWN",
        1: "UNCONFIGURED",
        2: "CONFIGURED",
        3: "RUNNING",
        4: "PAUSED",
        5: "ENDING",
        6: "STOPPING",
        7: "FAILING",
        8: "DONE",
        9: "STOPPED",
        10: "FAILED",
        11: "CONFIGURE_FAILED",
    }

    # Script path and salIndex we’re focusing on
    path = "maintel/m1m3/check_actuators.py"
    salIndex = 1

    df_script_logs = pd.DataFrame(script_logs)

    # Filter logs for the specific script path and salIndex
    df_filtered_logs = df_script_logs[
        (df_script_logs["path"] == path) & (df_script_logs["salIndex"] == salIndex)
    ]

    # Reindex by private_rcvStamp, convert to UTC
    df_filtered_logs = df_filtered_logs.set_index("private_rcvStamp")
    df_filtered_logs.index = pd.to_datetime(df_filtered_logs.index, unit="s")
    df_filtered_logs.index = df_filtered_logs.index - timedelta(seconds=37)

    # Convert all columns starting with 'timestamp' to datetime, also shift TAI->UTC
    timestamp_columns = [
        col for col in df_filtered_logs.columns if col.startswith("timestamp")
    ]
    for col in timestamp_columns:
        df_filtered_logs[col] = pd.to_datetime(df_filtered_logs[col], unit="s")
        df_filtered_logs[col] = df_filtered_logs[col] - timedelta(seconds=37)

    # Map numeric process/script states to strings
    df_filtered_logs["processState_str"] = df_filtered_logs["processState"].map(
        processState_mapping
    )
    df_filtered_logs["scriptState_str"] = df_filtered_logs["scriptState"].map(
        scriptState_mapping
    )

    # Remove unneeded columns
    columns_to_remove = [
        "blockId",
        "blockSize",
        "cmdId",
        "executionId",
        "private_efdStamp",
        "private_kafkaStamp",
        "private_origin",
        "private_revCode",
        "private_sndStamp",
        "private_seqNum",
        "scriptBlockIndex",
        "isStandard",
        "private_identity",
        "processState",
        "scriptState",
    ]
    df_filtered_logs.drop(columns=columns_to_remove, inplace=True, errors="ignore")

    # Sort from most recent to oldest
    df_filtered_logs.sort_index(ascending=False, inplace=True)

    return df_filtered_logs


def extract_execution_details(df_check_actuators_log):
    """
    Extracts execution details from the script logs DataFrame.

    Parameters
    ----------
    df_check_actuators_log : pd.DataFrame
        DataFrame containing the filtered script logs.

    Returns
    -------
    pd.DataFrame
        DataFrame containing execution times, durations, and final statuses.
    """
    # Initialize an empty list to store execution details
    execution_data = []

    # Loop over each unique scriptSalIndex
    for script_sal_index in df_check_actuators_log["scriptSalIndex"].unique():
        df_sal = df_check_actuators_log[
            df_check_actuators_log["scriptSalIndex"] == script_sal_index
        ]
        df_sal = df_sal.sort_index()  # Ensure chronological order

        # Calculate start and end times for each execution
        start_time = df_sal["timestampProcessStart"].min()
        end_time = df_sal["timestampProcessEnd"].max()

        # Get the final process and script status
        final_process_status = df_sal["processState_str"].iloc[-1]
        final_script_status = df_sal["scriptState_str"].iloc[-1]

        execution_data.append(
            {
                "scriptSalIndex": script_sal_index,
                "start_time": start_time,
                "end_time": end_time,
                "FinalProcessStatus": final_process_status,
                "FinalScriptStatus": final_script_status,
            }
        )

    # Create the DataFrame
    df_executions = pd.DataFrame(execution_data)

    if not df_executions.empty:
        # Calculate duration in minutes
        df_executions["duration_minutes"] = (
            df_executions["end_time"] - df_executions["start_time"]
        ).dt.total_seconds() / 60.0

        # Format durations to .2f
        df_executions["duration_minutes"] = df_executions["duration_minutes"].apply(
            lambda x: "{:.2f}".format(x)
        )

        # Reorder columns
        cols = [
            "scriptSalIndex",
            "start_time",
            "end_time",
            "duration_minutes",
            "FinalProcessStatus",
            "FinalScriptStatus",
        ]
        df_executions = df_executions[cols]

        # Sort by start time in descending order
        df_executions = df_executions.sort_values("start_time", ascending=False)
    else:
        print("No executions found.")

    return df_executions

In [ ]:
## Bump Test Queries and Processing

### Query Bump Logs


async def query_bump_logs(start_date: str, end_date: str, client_name="summit_efd"):
    """
    Queries the log messages related to bump tests from the EFD.

    Parameters
    ----------
    start_date : str
        Start date of the query in ISO format, e.g. "2024-11-04T12:00:00".
    end_date : str
        End date of the query in ISO format, e.g. "2024-11-04T13:00:00".
    client_name : str, optional
        Name of the EFD client. Defaults to "summit_efd".

    Returns
    -------
    pd.DataFrame
        DataFrame of bump log messages with the requested fields.
    """
    # Convert strings to datetime, then to astropy Time
    start_dt = datetime.fromisoformat(start_date)
    end_dt = datetime.fromisoformat(end_date)

    possible_clients = ["summit_efd", "usdf_efd"]
    if client_name not in possible_clients:
        print(f"Invalid client name. Possible clients: {possible_clients}")
        return None

    client = EfdClient(client_name)

    try:
        bump_logs = await client.select_time_series(
            topic_name="lsst.sal.MTM1M3.logevent_logMessage",
            fields=["message"],
            start=Time(start_dt.isoformat(), format="isot", scale="utc"),
            end=Time(end_dt.isoformat(), format="isot", scale="utc"),
        )
        return bump_logs
    except Exception as e:
        print(
            f"Error querying data from {start_dt.isoformat()} to {end_dt.isoformat()}: {e}"
        )
        return pd.DataFrame()


def process_bump_logs(bump_logs, expected_force_range=222, tolerance=5):
    """
    Processes bump log messages to extract relevant information and calculate deviations
    from expected forces.

    Parameters
    ----------
    bump_logs : pd.DataFrame
        DataFrame containing bump log messages.
    expected_force_range : float, optional
        Expected magnitude of the applied force.
    tolerance : float, optional
        Tolerance for the allowed variation in force.

    Returns
    -------
    pd.DataFrame
        Processed DataFrame with extracted and calculated data.
    """
    df_filtered_bump_log = bump_logs[
        bump_logs["message"].str.contains("Failed FA")
    ].copy()

    #    if df_filtered_bump_log.empty:
    #        print("No failed FA messages found in bump logs.")
    #        return pd.DataFrame()

    # Extract relevant info from the message
    df_filtered_bump_log.loc[:, "ID"] = df_filtered_bump_log["message"].str.extract(
        r"FA ID (\d+)"
    )
    orientation_index = df_filtered_bump_log["message"].str.extract(r"\((X|Y|Z)(\d+)\)")
    df_filtered_bump_log.loc[:, "Orientation"] = orientation_index[0]
    df_filtered_bump_log.loc[:, "Index"] = orientation_index[1]
    df_filtered_bump_log.loc[:, "Error Message"] = df_filtered_bump_log[
        "message"
    ].str.extract(r"- (.+)$")

    new_df = df_filtered_bump_log.reset_index().rename(columns={"index": "Time"})[
        ["Time", "ID", "Orientation", "Index", "Error Message"]
    ]

    # Extract measured force (ignore disabled actuators with zero force)
    new_df["MeasuredForce"] = (
        new_df["Error Message"].str.extract(r"\(([\-\d.]+)\)").astype(float)
    )
    new_df = new_df[new_df["MeasuredForce"] != 0]

    # Classify applied force direction
    new_df["AppliedForceDirection"] = new_df["Error Message"].apply(
        lambda x: "Positive" if "measured force plus" in x else "Negative"
    )

    # Calculate deviations
    upper_limit = expected_force_range
    lower_limit = -expected_force_range
    new_df["Deviation"] = 0.0

    for idx, row in new_df.iterrows():
        if row["AppliedForceDirection"] == "Positive":
            new_df.at[idx, "Deviation"] = float(row["MeasuredForce"]) - upper_limit
        elif row["AppliedForceDirection"] == "Negative":
            new_df.at[idx, "Deviation"] = float(row["MeasuredForce"]) - lower_limit

    # Classify deviation as overshoot or undershoot
    def classify_deviation(row):
        deviation = abs(row["MeasuredForce"]) - expected_force_range
        if deviation > 0:
            return "Overshoot"
        else:
            return "Undershoot"

    new_df["DeviationType"] = new_df.apply(classify_deviation, axis=1)

    cols = [
        "Time",
        "ID",
        "Orientation",
        "Index",
        "AppliedForceDirection",
        "MeasuredForce",
        "Deviation",
        "DeviationType",
    ]
    return new_df[cols]

In [ ]:
# Create a mapping from actuator_id to its index in FATable
m1m3_actuator_id_index_table = {fa.actuator_id: fa.index for fa in FATable}


def get_m1m3_actuator_ids():
    """Get a list of the M1M3 actuator ids."""
    return list(m1m3_actuator_id_index_table.keys())


def get_xy_position(actuator_list=FATable):
    xpos = [actuator.x_position for actuator in actuator_list]
    ypos = [actuator.y_position for actuator in actuator_list]
    return xpos, ypos


def ActuatorsLayout(ax, df, actuator_list=FATable):
    """
    Plot the layout of M1M3 actuators and highlight failed actuators.
    """
    orientation_colors = {"X": "blue", "Y": "red", "Z": "green"}
    combined_orientations_colors = {"XZ": "orange", "YZ": "black"}

    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("Failures Distribution", fontsize=12)

    ids = get_m1m3_actuator_ids()
    xpos, ypos = get_xy_position(actuator_list)

    actuator_orientations = defaultdict(set)
    for actuator_id, orientation in zip(df["ID"], df["Orientation"]):
        actuator_orientations[int(actuator_id)].add(orientation)

    ax.plot(xpos, ypos, "o", ms=14, color="blue", alpha=0.05, mec="red")
    for l, x, y in zip(ids, xpos, ypos):
        ax.annotate(
            l,
            (x, y),
            textcoords="offset points",
            xytext=(-5.5, -2),
            color="blue",
            size="xx-small",
        )

    for actuator_id, orientations in actuator_orientations.items():
        if actuator_id in m1m3_actuator_id_index_table:
            index = m1m3_actuator_id_index_table[actuator_id]
            if len(orientations) > 1:
                key = "".join(sorted(orientations))
                color = combined_orientations_colors.get(key, "gray")
            else:
                key = list(orientations)[0]
                color = orientation_colors.get(key, "gray")
            ax.scatter(
                xpos[index],
                ypos[index],
                marker="o",
                facecolors="none",
                edgecolors=color,
                s=250,
                alpha=0.5,
                linewidths=2,
            )

    # Legend
    unique_orientations = {
        ori
        for orientations in actuator_orientations.values()
        if len(orientations) == 1
        for ori in orientations
    }
    combined_orientations = {
        "".join(sorted(orientations))
        for orientations in actuator_orientations.values()
        if len(orientations) > 1
    }

    for orientation in unique_orientations:
        ax.scatter(
            [],
            [],
            marker="o",
            linestyle="None",
            s=10,
            facecolor="none",
            edgecolor=orientation_colors.get(orientation, "gray"),
            alpha=0.9,
            label=f"Orientation {orientation}",
        )

    for combined_orientation in combined_orientations:
        ax.scatter(
            [],
            [],
            marker="o",
            linestyle="None",
            s=10,
            facecolor="none",
            edgecolor=combined_orientations_colors.get(combined_orientation, "gray"),
            alpha=0.5,
            label=f"Orientation {combined_orientation}",
        )
    ax.legend(loc="upper left", bbox_to_anchor=(1, 1))


def plot_deviations_and_layout(
    df_failures, fig, actuator_list=FATable, expected_force=222, tolerance=5
):
    """
    Creates a single figure with:
      - Two subplots stacked vertically (Positive, Negative) on the left,
      - One subplot on the right for the actuator layout, with a square aspect.

    Assumes df_failures already has columns:
      'ID', 'DeviationType', 'AppliedForceDirection', 'MeasuredForce',
      and any others needed (e.g. 'Orientation').
    """

    if df_failures.empty:
        print("The DataFrame is empty. Nothing to plot.")
        return

    # Convert ID to string for plotting on x-axis
    df_failures["ID"] = df_failures["ID"].astype(str)
    sorted_ids = df_failures["ID"].sort_values().unique()

    # Dictionary for how each deviation type should look
    deviation_styles = {
        "Overshoot": {"color": "red", "marker": "o"},
        "Undershoot": {"color": "green", "marker": "s"},
    }

    # Map each ID to a position on the x-axis
    id_to_position = {id_: pos for pos, id_ in enumerate(sorted_ids)}
    df_failures["ID_Pos"] = df_failures["ID"].map(id_to_position)

    # Create the GridSpec layout within the provided figure
    gs = gridspec.GridSpec(
        nrows=2,
        ncols=2,
        width_ratios=[2, 1],  # left is twice as wide as right
        height_ratios=[2, 2],
        figure=fig,
    )

    # Left column subplots
    ax_positive = fig.add_subplot(gs[0, 0])
    ax_negative = fig.add_subplot(gs[1, 0], sharex=ax_positive)
    # Right column subplot (spans both rows)
    ax_layout = fig.add_subplot(gs[:, 1])

    # Separate the DataFrame
    df_positive = df_failures[df_failures["AppliedForceDirection"] == "Positive"]
    df_negative = df_failures[df_failures["AppliedForceDirection"] == "Negative"]

    # --- Subplot: Positive Forces ---
    for dev_type, group in df_positive.groupby("DeviationType"):
        style = deviation_styles.get(dev_type, {"color": "gray", "marker": "o"})
        ax_positive.scatter(
            group["ID_Pos"],
            group["MeasuredForce"],
            c=style["color"],
            marker=style["marker"],
            alpha=0.7,
            label=f"Positive - {dev_type}",
        )

    # Expected line & tolerance range for Positive
    ax_positive.axhline(expected_force, color="gray", linestyle="--", linewidth=0.8)
    ax_positive.fill_between(
        [-1, len(sorted_ids)],
        expected_force - tolerance,
        expected_force + tolerance,
        color="gray",
        alpha=0.2,
    )

    ax_positive.set_ylabel("Measured Force (N)")
    ax_positive.set_title("Measured Forces (Positive)")

    # Add ylim
    max_force = df_positive["MeasuredForce"].max()
    min_force = df_positive["MeasuredForce"].min()
    if max_force > expected_force + tolerance:
        ax_positive.set_ylim(top=max_force + 15)
    else:
        ax_positive.set_ylim(top=expected_force + tolerance + 15)

    if min_force < expected_force - tolerance:
        ax_positive.set_ylim(bottom=min_force - 15)
    else:
        ax_positive.set_ylim(bottom=expected_force - tolerance - 15)

    # --- Subplot: Negative Forces ---
    for dev_type, group in df_negative.groupby("DeviationType"):
        style = deviation_styles.get(dev_type, {"color": "gray", "marker": "o"})
        ax_negative.scatter(
            group["ID_Pos"],
            group["MeasuredForce"],
            c=style["color"],
            marker=style["marker"],
            alpha=0.7,
            label=f"Negative - {dev_type}",
        )

    ax_negative.axhline(-expected_force, color="gray", linestyle="--", linewidth=0.8)
    ax_negative.fill_between(
        [-1, len(sorted_ids)],
        -(expected_force + tolerance),
        -(expected_force - tolerance),
        color="gray",
        alpha=0.2,
    )

    ax_negative.set_xlabel("FA ID")
    ax_negative.set_ylabel("Measured Force (N)")
    ax_negative.set_title("Measured Forces (Negative)")

    # Add ylim like in the positive graph, but take into account that this in the negative side
    max_force = df_negative["MeasuredForce"].max()
    min_force = df_negative["MeasuredForce"].min()
    if max_force > -expected_force + tolerance:
        ax_negative.set_ylim(top=max_force + 15)
    else:
        ax_negative.set_ylim(top=-expected_force + tolerance + 15)

    if min_force < -expected_force - tolerance:
        ax_negative.set_ylim(bottom=min_force - 15)
    else:
        ax_negative.set_ylim(bottom=-expected_force - tolerance - 15)

    # Set x-ticks only on the bottom subplot
    ax_negative.set_xticks(range(len(sorted_ids)))
    ax_negative.set_xticklabels(sorted_ids, rotation=45, ha="center")

    # --- Build a custom legend for the top subplot ---
    legend_handles = []

    # 1) Add markers for Deviation Types
    for dev_type, style in deviation_styles.items():
        legend_handles.append(
            Line2D(
                [0],
                [0],
                marker=style["marker"],
                color="w",
                label=dev_type,
                markerfacecolor=style["color"],
                markersize=8,
            )
        )

    # 2) Add line handles for Expected Force + Tolerance
    expected_line = Line2D(
        [0],
        [0],
        color="gray",
        linestyle="--",
        label=f"Expected Force = {expected_force} ± {tolerance} N",
    )

    legend_handles.extend([expected_line])

    # Move legend outside of the graph and make it more compact
    ax_positive.legend(
        handles=legend_handles,
        loc="upper left",
        fontsize="small",
        borderaxespad=0,
        frameon=True,
    )

    # --- Right subplot: Layout of Actuators ---
    ActuatorsLayout(ax_layout, df_failures, actuator_list=actuator_list)

    # Ensure the layout subplot is square
    ax_layout.set_aspect("equal", adjustable="box")

    # Final adjustments
    plt.tight_layout()

## Getting Past Executions

### Script Queue Logs
Here we create a summary of the last few executions of the `check_actuators.py` script for the selected `past_time` and `client_name`.

In [ ]:
past_hours = convert_to_hours(past_time)

now_utc = datetime.utcnow()
end_dt = now_utc
start_dt = now_utc - timedelta(hours=past_hours)

start_str = start_dt.isoformat()
end_str = end_dt.isoformat()

script_logs = await query_script_queue_logs(start_str, end_str, client_name=client_name)
script_logs_processed = filter_and_process_queue_logs(script_logs)
df_executions = extract_execution_details(script_logs_processed)

# Display executions summary
if df_executions.empty:
    print("No executions found in the last {:.1f} days.".format(past_hours / 24.0))
else:
    print("Executions found in the last {:.1f} days.".format(past_hours / 24.0))
    display(df_executions)

### Interactive Bokeh App to Query and Plot Bump Test Failures

In this section, an interactive **Bokeh-based app** is provided to guide you through the bump test log analysis:

1. **`SalIndex` Dropdown:**
   Allows you to select a specific script execution (identified by its `SalIndex`) within the provided time range.
   Once selected, the app retrieves the associated execution details (start time, end time, duration, etc.).
   Only executions within the selected time range are displayed.

2. **Execution Summary:**
   Upon selecting a `SalIndex`, the notebook displays detailed execution information, including:
   - Start and end times of the execution.
   - Execution duration.
   - Process and script statuses.

3. **Failure Summary Table:**
   If failed actuators are detected, the app shows a table summarizing:
   - The time of failure.
   - Actuator ID.
   - Orientation and force details (measured force, deviation, and deviation type).
   - A clickable link to detailed plots for individual actuators.

4. **“Plot Measured Forces” Checkbox:**
   Toggles the visualization of measured forces for all failed actuators. When enabled, the notebook displays:
   - A set of plots showing positive and negative force deviations for the failed actuators.
   - A layout plot indicating the spatial distribution of failed actuators on the M1M3 system.

5. **Dynamic Visualization:**
   The app dynamically updates the table, execution summary, and plots based on the selected `SalIndex` and the checkbox state.
   This streamlined interface allows users to quickly explore and analyze bump test logs.

---

**<span style="color: red;">Note:</span>**
The expected measured force for each bump test is **222 N** with a tolerance of **±5 N**. Deviations beyond this range are classified as either **overshoot** or **undershoot** based on the measured force.


In [ ]:
# Define helper functions, build_all_data, and launch Bokeh app

# Initialize Bokeh to display plots in the notebook
output_notebook()

# ---------------------------
# Define Helper Functions
# ---------------------------


def make_fa_link(fa_id, t_start_str, t_end_str):
    """
    Create an HTML link to the detail notebook for a given FA ID.

    Parameters:
    - fa_id: int or str, Force Actuator ID.
    - t_start_str: str, ISO-formatted start time.
    - t_end_str: str, ISO-formatted end time.

    Returns:
    - str: HTML <a> tag linking to the detail notebook.
    """
    detail_url = (
        "https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst-sitcom/"
        "reports-performance-summary/sst/mtm1m3/Bump_test_individual_force_actuator"
        f"?t_start={t_start_str}&t_end={t_end_str}&id={fa_id}&ts_hide_code=1"
    )
    return f'<a href="{detail_url}" target="_blank">{fa_id}</a>'


def mpl_fig_to_base64_png(fig):
    """
    Convert a Matplotlib figure to a base64-encoded PNG <img> tag.
    """
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    b64 = base64.b64encode(buf.read()).decode("utf-8")
    buf.close()  # Close the buffer to release resources
    return b64


# ---------------------------
# Define build_all_data Function
# ---------------------------


async def build_all_data(df_executions, client_name):
    """
    For each SalIndex in df_executions:
      1) Query bump logs.
      2) Process bump logs.
      3) Create clickable links for FA IDs.
      4) Generate Matplotlib plots and convert to base64.

    Parameters:
    - df_executions: pandas DataFrame with execution details.
    - client_name: str, name of the EFD client.

    Returns:
    - dict: Keyed by SalIndex, each containing info_html, df_html, plot_html.
    """
    all_data = {}

    for _, row in df_executions.iterrows():
        sal_idx = row["scriptSalIndex"]
        t_start = row["start_time"]
        t_end = row["end_time"]
        duration = row["duration_minutes"]
        final_proc = row["FinalProcessStatus"]
        final_script = row["FinalScriptStatus"]

        start_str = t_start.isoformat()
        end_str = t_end.isoformat()

        # Adjust start time by subtracting 10 days for the link
        adjusted_start = Time(t_start).datetime - timedelta(days=10)
        adjusted_start_str = adjusted_start.isoformat()

        # 1) Query bump logs
        bump_logs = await query_bump_logs(start_str, end_str, client_name)
        if bump_logs.empty:
            all_data[sal_idx] = {
                "info_html": (
                    f"<b>SalIndex {sal_idx}</b><br>"
                    f"No bump logs found.<br>"
                    f"Start: {t_start}, End: {t_end}, Duration: {duration} min"
                ),
                "df_html": "<p>No logs found.</p>",
                "plot_html": "",
            }
            continue

        # 2) Process bump logs
        df_fail = process_bump_logs(bump_logs)
        if df_fail.empty:
            all_data[sal_idx] = {
                "info_html": (
                    f"<b>SalIndex {sal_idx}</b><br>"
                    f"No failed actuators found.<br>"
                    f"Start: {t_start}, End: {t_end}, Duration: {duration} min"
                ),
                "df_html": "<p>No failed actuators.</p>",
                "plot_html": "",
            }
            continue

        # 3) Create clickable links for FA IDs
        if "ID" in df_fail.columns and "Orientation" in df_fail.columns:
            df_fail["Force_Error_Plots_Link"] = df_fail.apply(
                lambda r: make_fa_link(r["ID"], adjusted_start_str, end_str), axis=1
            )
            # Ensure 'ID' remains for plotting
        else:
            df_fail["Force_Error_Plots_Link"] = ""

        # 4) Convert DataFrame to HTML with clickable links
        df_html = df_fail.to_html(escape=False, index=False)

        # 5) Generate the plot and convert to base64
        fig = plt.figure(figsize=(14, 5))
        plot_deviations_and_layout(df_fail, fig, actuator_list=FATable)
        plot_html = mpl_fig_to_base64_png(fig)
        plt.close(fig)  # Prevent automatic display

        # 6) Create informational HTML
        info_html = (
            f"<b>SalIndex {sal_idx}</b><br>"
            f"Start: {t_start}, End: {t_end}, Duration: {duration} min<br>"
            f"ProcessStatus: {final_proc}, ScriptStatus: {final_script}"
        )

        # 7) Store in the dictionary
        all_data[sal_idx] = {
            "info_html": info_html,
            "df_html": df_html,
            "plot_html": plot_html,
        }

    return all_data


# ---------------------------
# Build all_results by Processing df_executions
# ---------------------------

# Execute the build_all_data function to populate all_results
all_results = await build_all_data(df_executions, client_name)

# ---------------------------
# Convert all_results to a DataFrame for Bokeh
# ---------------------------

if all_results:
    df_bokeh = pd.DataFrame(
        [
            {
                "SalIndex": sal_idx,
                "info_html": data["info_html"],
                "df_html": data["df_html"],
                "plot_html": data["plot_html"],
            }
            for sal_idx, data in all_results.items()
        ]
    )
else:
    # Handle the case where all_results is empty
    df_bokeh = pd.DataFrame(columns=["SalIndex", "info_html", "df_html", "plot_html"])

# ---------------------------
# Create a ColumnDataSource from the DataFrame
# ---------------------------

source = ColumnDataSource(df_bokeh)

# ---------------------------
# Define the Bokeh Widgets
# ---------------------------

# Define the SalIndex dropdown using Bokeh's Select widget
sal_index_selector = Select(
    title="SalIndex:",
    value=df_bokeh["SalIndex"].iloc[-1]
    if not df_bokeh.empty
    else None,  # Default to the last SalIndex
    options=[
        (sal_idx, f"{sal_idx}") for sal_idx in sorted(df_bokeh["SalIndex"].tolist())
    ],
    width=200,
)

# Define the checkbox to toggle plot visibility using Bokeh's Checkbox widget
show_forces_checkbox = Checkbox(label="Plot measured forces", active=True, width=200)

# Define Div widgets for displaying informational text, summary table, and plot
info_div = Div(text="<b>Select a SalIndex to see details.</b>", width=800, height=100)

table_div = Div(
    text="", width=1200, styles={"overflow": "auto", "padding": "10px"}
)  # Initially empty

plot_div = Div(
    text="", styles={"overflow": "auto", "padding": "10px"}
)  # Initially empty

# ---------------------------
# Define the CustomJS Callbacks
# ---------------------------

# Define the CustomJS callback for the SalIndex Select widget
select_callback = CustomJS(
    args=dict(
        source=source,
        info_div=info_div,
        table_div=table_div,
        plot_div=plot_div,
        checkbox=show_forces_checkbox,
    ),
    code="""
    // Get the selected SalIndex
    var data = source.data;
    var sal_idx = cb_obj.value;
    var show_plot = checkbox.active;

    // Find the index of the selected SalIndex
    var index = data['SalIndex'].indexOf(sal_idx);

    if(index === -1){
        // If SalIndex not found, clear the Divs
        info_div.text = "<b>Select a SalIndex to see details.</b>";
        table_div.text = "";
        plot_div.text = "";
    }
    else{
        // Update the informational Div
        info_div.text = data['info_html'][index];

        // Update the summary table Div
        table_div.text = data['df_html'][index];

        // Update the plot Div based on the checkbox state
        if(show_plot && data['plot_html'][index]){
            plot_div.text = "<img src='data:image/png;base64," + data['plot_html'][index] + "'/>";
        }
        else{
            plot_div.text = "";
        }
    }
""",
)

# Define the CustomJS callback for the Checkbox widget
checkbox_callback = CustomJS(
    args=dict(
        source=source,
        info_div=info_div,
        table_div=table_div,
        plot_div=plot_div,
        sal_select=sal_index_selector,
    ),
    code="""
    // Get the current SalIndex
    var data = source.data;
    var sal_idx = sal_select.value;
    var show_plot = cb_obj.active;

    // Find the index of the selected SalIndex
    var index = data['SalIndex'].indexOf(sal_idx);

    if (index === -1) {
        // If SalIndex not found, do nothing
        return;
    } else {
        // Update the plot Div based on the checkbox state
        if (show_plot && data['plot_html'][index]) {
            // Ensure proper formatting of the <img> tag
            plot_div.text = "<img src='data:image/png;base64," + data['plot_html'][index].trim() +
                            "' style='max-width: 80%; height: auto; display: block; margin: 0 auto;' />";
        } else {
            plot_div.text = "";
        }

        // Adjust spacing
        plot_div.style.marginTop = "10px";
        plot_div.style.marginBottom = "0px";
    }
""",
)

# ---------------------------
# Attach the Callbacks to the Widgets
# ---------------------------

# Attach the CustomJS callback to the SalIndex Select widget
sal_index_selector.js_on_change("value", select_callback)

# Attach the CustomJS callback to the Checkbox widget
show_forces_checkbox.js_on_change("active", checkbox_callback)

# ---------------------------
# Initialize the Display
# ---------------------------


def initialize_display():
    """
    Initializes the Div widgets based on the current widget states.
    """
    if not df_bokeh.empty and sal_index_selector.value is not None:
        sal_idx = sal_index_selector.value
        index = df_bokeh["SalIndex"].tolist().index(sal_idx)
        info_div.text = df_bokeh["info_html"].iloc[index]
        table_div.text = df_bokeh["df_html"].iloc[index]
        if show_forces_checkbox.active and df_bokeh["plot_html"].iloc[index]:
            plot_div.text = f"<img src='data:image/png;base64,{df_bokeh['plot_html'].iloc[index]}'/>"
    else:
        info_div.text = "<b>Select a SalIndex to see details.</b>"
        table_div.text = ""
        plot_div.text = ""


# Call the initialization function to set up the initial display
initialize_display()

# ---------------------------
# Arrange and Display the Layout
# ---------------------------

# Arrange the widgets and Divs in the layout
layout = column(
    row(sal_index_selector, show_forces_checkbox),
    info_div,
    table_div,
    plot_div,
    spacing=0,  # Reduce the default spacing
    sizing_mode="stretch_width",  # Optional: Adjust width dynamically
)

# Display the layout
show(layout)